In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Data/Train_NY_cleaned.csv')
dft= pd.read_csv('/content/drive/MyDrive/Data/Test_NY_cleaned.csv')
df.drop({'Unnamed: 0'}, axis=1, inplace=True)
dft.drop({'Unnamed: 0'}, axis=1, inplace=True)

In [ ]:
df.head(1)

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,BLOCK,LOT,ZIP CODE,RESIDENTIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,1,ALPHABET CITY,RENTALS - WALKUP APARTMENTS,392,6,10009,5,5,1633.0,6440.0,1900,2,C2,6625000.0,2017-07-19


In [ ]:
df.loc[df['NEIGHBORHOOD'] == 'ALPHABET CITY']['ZIP CODE'].unique()

array([10009, 10002])

In [ ]:
alph = pd.Series(df.loc[df['NEIGHBORHOOD']== 'TRAVIS']['SALE PRICE'])
alph

In [ ]:
chel = pd.Series(df.loc[df['NEIGHBORHOOD']== 'WEST NEW BRIGHTON']['SALE PRICE'])
chel

In [ ]:
chel2= chel.sample(71)
import scipy.stats as stats

In [ ]:
stats.ttest_ind(alph, chel)

Ttest_indResult(statistic=3.3722846815724576, pvalue=0.0008422520672824367)

In [ ]:
df['SALE DATE'] = pd.to_datetime(df['SALE DATE'])

# Linear Regression

### With dummies

In [ ]:
df.columns 

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY', 'BLOCK', 'LOT',
       'ZIP CODE', 'RESIDENTIAL UNITS', 'TOTAL UNITS', 'LAND SQUARE FEET',
       'GROSS SQUARE FEET', 'YEAR BUILT', 'TAX CLASS AT TIME OF SALE',
       'BUILDING CLASS AT TIME OF SALE', 'SALE PRICE', 'SALE DATE'],
      dtype='object')

In [ ]:
dum = df[['BOROUGH','NEIGHBORHOOD', 'BLOCK', 'LOT',
       'ZIP CODE', 'RESIDENTIAL UNITS', 'TOTAL UNITS', 'LAND SQUARE FEET',
       'GROSS SQUARE FEET', 'YEAR BUILT', 'TAX CLASS AT TIME OF SALE', 'SALE PRICE']]

In [ ]:
dum = pd.get_dummies(dum, columns=['NEIGHBORHOOD'], drop_first=True)
y= dum['SALE PRICE']
dum.drop({'SALE PRICE'}, axis=1, inplace=True)

from sklearn.linear_model import  LinearRegression
lr= LinearRegression()

from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(dum, y)

print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)

(44819, 263)
(44819,)
(14940, 263)


In [ ]:
lr.fit(xtrain, ytrain)
print(lr.score(xtrain,ytrain))
print(lr.score(xtest,ytest))

0.33534935449854175
0.34107878361528143


In [ ]:
# Standardization
from sklearn.preprocessing import StandardScaler
st = StandardScaler()

stdf = st.fit_transform(dum)
xtrain, xtest, ytrain, ytest = train_test_split(stdf, y)

In [ ]:
lr= LinearRegression()
lr.fit(xtrain, ytrain)
print(lr.score(xtrain,ytrain))
print(lr.score(xtest,ytest))

0.4356358078099348
0.19707016847204928


### With Target Encoding

In [ ]:
!pip install category_encoders
from category_encoders import TargetEncoder
encoder = TargetEncoder()

In [ ]:
df1 = df.copy()
df.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY', 'BLOCK', 'LOT',
       'ZIP CODE', 'RESIDENTIAL UNITS', 'TOTAL UNITS', 'LAND SQUARE FEET',
       'GROSS SQUARE FEET', 'YEAR BUILT', 'TAX CLASS AT TIME OF SALE',
       'BUILDING CLASS AT TIME OF SALE', 'SALE DATE'],
      dtype='object')

In [ ]:
y= df['SALE PRICE']
df.drop({'SALE PRICE'}, axis=1, inplace=True)

In [ ]:
training_set = encoder.fit_transform(df, y)

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [ ]:
training_set.drop({'SALE DATE'}, axis=1, inplace=True)

In [ ]:
st = StandardScaler()

stdf = st.fit_transform(training_set)
xtrain, xtest, ytrain, ytest = train_test_split(stdf, y)

In [ ]:
lr= LinearRegression()
lr.fit(xtrain, ytrain)
print(lr.score(xtrain,ytrain))
print(lr.score(xtest,ytest))

0.4994657300414437
0.38288982820188644


### Label Encoding

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
df['NEIGHBORHOOD'] = le.fit_transform(df['NEIGHBORHOOD'])

In [ ]:
df['BUILDING CLASS CATEGORY'] = le.fit_transform(df['BUILDING CLASS CATEGORY']) # BUILDING CLASS AT TIME OF SALE
df['BUILDING CLASS AT TIME OF SALE'] = le.fit_transform(df['BUILDING CLASS AT TIME OF SALE'])

In [ ]:
df.drop({'SALE DATE'}, axis=1, inplace=True)

In [ ]:
st = StandardScaler()

stdf = st.fit_transform(df)
xtrain, xtest, ytrain, ytest = train_test_split(stdf, y)

In [ ]:
lr= LinearRegression()
lr.fit(xtrain, ytrain) # Regularization and K Fold cross validation 
print(lr.score(xtrain,ytrain))
print(lr.score(xtest,ytest))

0.3994023929084529
0.09464449448072931


# Lasso

In [ ]:
# With dummies
dum = df1[['BOROUGH','NEIGHBORHOOD', 'BLOCK', 'LOT',
       'ZIP CODE', 'RESIDENTIAL UNITS', 'TOTAL UNITS', 'LAND SQUARE FEET',
       'GROSS SQUARE FEET', 'YEAR BUILT', 'TAX CLASS AT TIME OF SALE']]

dum = pd.get_dummies(dum, columns=['NEIGHBORHOOD'], drop_first=True)

st = StandardScaler()

stdf = st.fit_transform(dum)
xtrain, xtest, ytrain, ytest = train_test_split(stdf, y)

In [ ]:
from sklearn.linear_model import Lasso
lr= Lasso()
lr.fit(xtrain, ytrain)
print(lr.score(xtrain,ytrain))
print(lr.score(xtest,ytest))

0.4435568318368758
-0.08595748167985051


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.401520354769963e+18, tolerance: 864329286338053.9
  positive)


In [ ]:
# Target Encoding:

stdf = st.fit_transform(training_set)
xtrain, xtest, ytrain, ytest = train_test_split(stdf, y)

In [ ]:
lr= Lasso()
lr.fit(xtrain, ytrain)
print(lr.score(xtrain,ytrain))
print(lr.score(xtest,ytest))

0.39647681648174526
0.3944479790625926


In [ ]:
# Label Encoding

In [ ]:
stdf = st.fit_transform(df)
xtrain, xtest, ytrain, ytest = train_test_split(stdf, y)

In [ ]:
lr= Lasso()
lr.fit(xtrain, ytrain)
print(lr.score(xtrain,ytrain))
print(lr.score(xtest,ytest))

0.37931964596282963
0.1895609852489619


# Ridge

In [ ]:
# Ridge Regression

from sklearn.linear_model import RidgeCV
lr = RidgeCV(cv=5)

In [ ]:
#  With dummies


dum = df1[['BOROUGH','NEIGHBORHOOD', 'BLOCK', 'LOT',
       'ZIP CODE', 'RESIDENTIAL UNITS', 'TOTAL UNITS', 'LAND SQUARE FEET',
       'GROSS SQUARE FEET', 'YEAR BUILT', 'TAX CLASS AT TIME OF SALE']]

dum = pd.get_dummies(dum, columns=['NEIGHBORHOOD'], drop_first=True)

st = StandardScaler()

stdf = st.fit_transform(dum)
xtrain, xtest, ytrain, ytest = train_test_split(stdf, y)

In [ ]:
lr.fit(xtrain, ytrain)
print(lr.score(xtrain,ytrain))
print(lr.score(xtest,ytest))

0.33793112826725735
0.37490525234181415


In [ ]:
# With Target Encoding

stdf = st.fit_transform(training_set)
xtrain, xtest, ytrain, ytest = train_test_split(stdf, y)

lr = RidgeCV(cv=5)
lr.fit(xtrain, ytrain)
print(lr.score(xtrain,ytrain))
print(lr.score(xtest,ytest))

0.457908217466941
0.5048322576926769


In [ ]:
# With Label Encoding:

stdf = st.fit_transform(df)
xtrain, xtest, ytrain, ytest = train_test_split(stdf, y)

lr = RidgeCV(cv=5)
lr.fit(xtrain, ytrain)
print(lr.score(xtrain,ytrain))
print(lr.score(xtest,ytest))

0.39679104272105403
-0.14349709662260812


In [ ]:
# Unsupervised 
||
# Supervised

SyntaxError: ignored